In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from MIOFlow.utils import generate_steps, set_seeds, config_criterion
from MIOFlow.models import make_model, GrowthRateModel
from MIOFlow.plots import plot_comparision, plot_losses
from MIOFlow.train import train_ae, training_regimen
from MIOFlow.constants import ROOT_DIR, DATA_DIR, NTBK_DIR, IMGS_DIR, RES_DIR
from MIOFlow.datasets import (
    make_diamonds, make_swiss_roll, make_tree, make_eb_data, 
    make_dyngen_data
)
from MIOFlow.geo import setup_distance
from MIOFlow.exp import setup_exp
from MIOFlow.eval import generate_plot_data

import os, pandas as pd, numpy as np, \
    seaborn as sns, matplotlib as mpl, matplotlib.pyplot as plt, \
    torch, torch.nn as nn

from tqdm.notebook import tqdm
import time

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Load dataset

In [3]:
#| export
def branch_data_clean(t, e):
    shapes = (len(e), len(t))
    t = t.reshape(1,-1)
    e = e.reshape(-1, 1)
    ts = np.tile(t, (shapes[0], 1))[...,None]
    es = np.tile(e, (1, shapes[1]))[...,None]
    x = np.tile(t, (shapes[0], 1))
    y = e * t**2
    data = np.stack([x,y], axis=2)
    data = np.concatenate((ts, es, data), axis=-1)
    data = data.reshape(shapes[0]*shapes[1], -1)
    return data
def branch_data_data(data, n_colors=5, repeats=5, noisex=0.05, noisey=0.05, seed=32):
    data = np.tile(data, (repeats,1))
    df = pd.DataFrame(data, columns=['t', 'e1', 'd1', 'd2'])
    np.random.seed(seed)
    df['d1'] += np.random.randn(df.shape[0]) * noisex
    df['d2'] += np.random.randn(df.shape[0]) * noisey
    _, bin_edges = np.histogram(df['t'], bins=n_colors)
    bin_indices = np.digitize(df['t'], bin_edges)
    bin_indices[bin_indices > n_colors] = n_colors
    df.insert(0, 'samples', bin_indices)
    df.drop(columns=['t'], inplace=True)
    df.samples -=1
    return df
def make_branch_cond(nt=20, ne=3, n_colors=5, repeats=10, noisex=0.1, noisey=0.1, seed=32):
    t = np.linspace(0,1,nt)
    e = np.linspace(-1,1,ne)
    data = branch_data_clean(t, e)
    df = branch_data_data(data, n_colors, repeats, noisex, noisey, seed)
    return df

df = make_branch_cond(ne=2, repeats=20)

df = df[['samples', 'd1', 'd2', 'e1']]

In [ ]:
df

In [ ]:
fig = plt.figure(figsize=(14, 8), dpi=300)
sns.scatterplot(data=df, x='d1', y='d2', hue='samples', palette='viridis');

In [ ]:
fig = plt.figure(figsize=(14, 8), dpi=300)
sns.scatterplot(data=df, x='d1', y='d2', hue='e1', palette='viridis');

In [7]:
df_copy = df.copy()
# df = df.drop(columns=['e1'])

In [8]:
means = df[['d1', 'd2']].mean(axis=0)
stds = df[['d1', 'd2']].std(axis=0)
df[['d1', 'd2']] = (df[['d1', 'd2']] - means) / stds

# Train autoencoder or the geodesic embedding

#### Set seeds and check GPU

In [9]:
set_seeds(0)
use_cuda = torch.cuda.is_available()

#### Handle hold-out training condition

In [10]:
# This is True if we want to holdout (or skip) one timepoint during training. It is used to test the accuracy of the trajectories on unseen data.
hold_one_out = False
# It can be a group number or 'random', works in tandem with hold_one_out
hold_out = 3

# The dimensions in the input space, it is columns - 1 because we assume one column is equal to "samples".
model_features = len(df.columns) - 1 - 1 # drop the condition
groups = sorted(df.samples.unique())

In [11]:
# # These determine the logic flow for training: 
# #   use_emb=True use_gae=False is only the encoder to match the approximation of the geodesic.
# #   use_emb=False use_gae=True the full Geodesic Autoencoder (GAE), i.e. matching the geodesic and a reconstruction loss.
# #   use_emb=False use_gae=False Is not using the GAE.
# #   use_emb=True use_gae=True, is redundant and should raise an error. 
# use_emb = False
# use_gae = False

# need_to_train_gae = (use_emb or use_gae) and use_emb != use_gae

# # If the reconstruction loss needs to be computed.
# recon = use_gae and not use_emb 

# # These are training GAE hyperparameters needed for training
# # Distance_type in ['gaussian', 'alpha_decay'], and Gaussian scale
# distance_type = 'gaussian'
# rbf_length_scale=0.1
# dist = setup_distance(distance_type, rbf_length_scale=rbf_length_scale)

# #Can be changed depending on the dataset
# n_epochs_emb = 1000
# samples_size_emb = (30, )

# # Layers for the Geodesic Autoencoder
# gae_embedded_dim = 3
# encoder_layers = [model_features, 8, gae_embedded_dim]

# gae = Autoencoder(
#     encoder_layers = encoder_layers,
#     decoder_layers = encoder_layers[::-1],
#     activation='ReLU', use_cuda = use_cuda
# )
# optimizer = torch.optim.AdamW(gae.parameters())

# Specify parameters

In [12]:
set_seeds(10)

#Directory where results are saved
exp_name = 'test'

# density loss knn
use_density_loss = True

# Weight of density (not percentage of total loss)
# lambda_density = 35
lambda_density = 0

# For petal=LeakyReLU / dyngen=CELU
activation = 'LeakyReLU'

# Can change but we never really do, mostly depends on the dataset.
layers = [16,32,16]

# Scale of the noise in the trajectories. Either len(groups)*[float] or None. Should be None if using an adaptative ODE solver.
sde_scales = len(groups)*[0.1] 


# model = make_model(
#     model_features, layers, which='sde', method='euler',
#     activation=activation, scales=sde_scales, use_cuda=use_cuda
# )

# model = make_model(
#     model_features, layers, which='sde', method='reversible_heun', sde_type='stratonovich', adjoint_method='adjoint_reversible_heun',
#     activation=activation, scales=sde_scales, use_cuda=use_cuda, use_norm=False
# )


model = make_model(
    model_features, layers, which='sde', method='euler', sde_type='ito', 
    activation=activation, scales=sde_scales, use_cuda=use_cuda, use_norm=True, n_conditions=1, momentum_beta=0.5
)

In [13]:
# Basically "batch size"
# sample_size=(60, )
sample_size=(60, )

# Training specification
n_local_epochs = 40
n_epochs = 0
n_post_local_epochs = 0

# Using the reverse trajectories to train
reverse_schema = True
# each reverse_n epoch
reverse_n = 2


criterion_name = 'ot'
criterion = config_criterion(criterion_name, covariance_lambda=0.0, detach_dist_for_plan=False, detach_mass=False)

optimizer = torch.optim.AdamW(model.parameters())

# Bookkeeping variables
batch_losses = []
globe_losses = []
if hold_one_out and hold_out in groups:
    local_losses = {f'{t0}:{t1}':[] for (t0, t1) in generate_steps(groups) if hold_out not in [t0, t1]}
else:
    local_losses = {f'{t0}:{t1}':[] for (t0, t1) in generate_steps(groups)}

# For creating output.
n_points = 100
n_trajectories = 100
n_bins = 100


In [14]:
exp_dir = './results/test_all/'

In [ ]:
df

In [ ]:
gr_model = GrowthRateModel(
    feature_dims=2,
    condition_dims=1,
    layers=[64,64],
    activation='LeakyReLU',
    use_time=True,
)


In [ ]:
start_time = time.time()
local_losses, batch_losses, globe_losses = training_regimen(
    # local, global, local train structure
    n_local_epochs=n_local_epochs, 
    n_epochs=n_epochs, 
    n_post_local_epochs=n_post_local_epochs,
    
    # where results are stored
    exp_dir=exp_dir, 

    # BEGIN: train params
    model=model, df=df, groups=groups, optimizer=optimizer, 
    criterion=criterion, use_cuda=use_cuda,
    
    hold_one_out=hold_one_out, hold_out=hold_out,
    
    use_density_loss=use_density_loss, 
    lambda_density=0.2,
    # lambda_density=10,
    
    autoencoder=None, use_emb=False, use_gae=False, 
    
    sample_size=sample_size, logger=None,
    reverse_schema=False, reverse_n=reverse_n, # sde cant use reverse schema for now
    # use_penalty=True, 
    use_penalty=True, 
    # lambda_energy=0.001,
    lambda_energy_local = {g:1e-3 for g in groups},
    lambda_energy_global = {g:1e-3 for g in groups},
    # END: train params

    plot_every=5,
    n_points=n_points, n_trajectories=n_trajectories, n_bins=n_bins, 
    #local_losses=local_losses, batch_losses=batch_losses, globe_losses=globe_losses
    lambda_cond=1.,
)
# run_time = time.time() - start_time + run_time_geo if use_emb or use_gae else time.time() - start_time
# logger.info(f'Total run time: {np.round(run_time, 5)}')

In [ ]:
model

In [ ]:
plot_losses(
    local_losses, batch_losses, globe_losses, 
    save=True, path=exp_dir, file='losses.png'
)

In [ ]:
df

In [ ]:
groups

: 

In [ ]:
data_t0

: 

In [ ]:
from MIOFlow.utils import sample


data_t0 = sample(df, np.min(groups), replace=False, to_torch=True, use_cuda=use_cuda)

sample_time = np.linspace(np.min(groups), np.max(groups), n_bins)
sample_time = torch.tensor(sample_time, dtype=data_t0.dtype, device=data_t0.device)

with torch.no_grad():
    x_traj = model(data_t0, sample_time, return_whole_sequence=True)
    # x_trajs.append(x_traj)

fig, ax = plt.subplots(figsize=(8, 8))

scatter = ax.scatter(df['d1'], df['d2'],
                c=df['samples'], cmap='viridis', s=10, alpha=.5)

ax.set_xticks([])
ax.set_yticks([])

for i,traj in enumerate(np.transpose(x_traj, axes=(1,0,2))):
    ax.plot(traj[:, 0], traj[:, 1], alpha=.5, linewidth=.5, color='Black')
    ax.annotate('', xy=(traj[-1, 0], traj[-1, 1]), xytext=(traj[-2, 0], traj[-2, 1]),
                    arrowprops=dict(arrowstyle="-|>", color='Black', lw=1))
ax.set_title('Trajectory')

In [ ]:
model.eval()
groups = sorted(df['samples'].unique())
sample_time = np.linspace(np.min(groups), np.max(groups), n_bins)
from MIOFlow.utils import sample
np.random.seed(32)
data_t0 = sample(
    df, np.min(groups), size=(n_points, ), 
    replace=False, to_torch=True, use_cuda=use_cuda
)
# data_t0 = torch.tensor([[-1., 0.]], dtype=torch.float32, device='cpu')
fig, ax = plt.subplots(figsize=(5, 5))

sample_time = torch.tensor(sample_time, dtype=data_t0.dtype, device=data_t0.device)

with torch.no_grad():
    x_traj = model(data_t0, sample_time, return_whole_sequence=True)

scatter = ax.scatter(df['d1'], df['d2'],
                c=df['samples'], cmap='viridis', s=10, alpha=.5)

ax.set_xticks([])
ax.set_yticks([])

for i,traj in enumerate(np.transpose(x_traj, axes=(1,0,2))):
    ax.plot(traj[:, 0], traj[:, 1], alpha=.5, linewidth=.5, color='Black')
    ax.annotate('', xy=(traj[-1, 0], traj[-1, 1]), xytext=(traj[-2, 0], traj[-2, 1]),
                 arrowprops=dict(arrowstyle="-|>", color='Black', lw=1))

plt.tight_layout()

In [ ]:
data_t0